# Week H

More Neural Networks

In [ ]:
import torch
import matplotlib.pyplot as plt

## More Tensors and Why They're Awesome

Multi-dimensional slicing is definitely a nice property of tensors, but what really sets them apart is their ability to keep track of all the operations performed on them using _computational graphs_.

If we define a tensor and set its `requires_grad` parameter to `True` we unlock some really nice properties that we can use for training neural networks.

One of these properties is the ability to automatically calculate derivatives (OMG, calculus!) of functions defined in terms of our tensor.

Let's investigate.

### Easy Calculus and Free Derivatives

Let's pretend we have the following function:

$f(x) = x^4 - 0.7x^3 - 2x^2 + x + 1$

And we want to find out when the function achieves its maximum and minimum values, when it equals $0$, or when it equals $0.5$.

We can plot it, and easily approximate those values visually:

In [ ]:
def peaks(x):
  return x**4 - 0.7*x**3 - 2*x**2 + x + 1

In [ ]:
# linspace is range()'s cousin, but for floats 
#   and where the 3rd argument specifies number of steps, not length of steps

x = torch.linspace(-1.3, 1.6, 300)
y = peaks(x)

plt.plot(x, y)
plt.plot([-1.3, 1.6], [0,0], '-')
plt.plot([-1.3, 1.6], [0.5, 0.5], '-')
plt.show()

Looks like local minimum and maximum values are approximately:
- $x = -0.9$ (global minimum)
- $x = 0.2$ (global maximum)
- $x = 1.2$ (local minimum)

It crosses $y = 0$ at:
- $x = -1.2$
- $x = -0.6$

And, it crosses $y=0.5$ a bunch of times, so we'll look at that later.

We can calculate exact values for these points in our graph if we define $x$ and $y$ as tensors and enable their `auto_grad` functionality.

In [ ]:
xt = torch.linspace(-1.3, 1.6, 8000, requires_grad=True)
yt = peaks(xt)
yt.backward(torch.ones_like(xt))

dydx = xt.grad
print("derivatives:", dydx[:5])

minmax_idx = (dydx.abs() < 9e-4)
minmax_y = yt[minmax_idx]
minmax_x = xt[minmax_idx]

plt.plot(x, y)
plt.plot(minmax_x.tolist(), minmax_y.tolist(), 'o')
plt.show()

print("min/max:", minmax_x, minmax_y)

In [ ]:
xt = torch.linspace(-1.3, 1.6, 8000, requires_grad=True)
yt = peaks(xt).pow(2)
yt.backward(torch.ones_like(xt))

dydx = xt.grad
print("derivatives:", dydx[:5])

zeros_idx = ((dydx.abs() < 0.005) & (yt < 1e-7))
zeros_x = xt[zeros_idx]
zeros_y = yt[zeros_idx]

plt.plot(x, y)
plt.plot(zeros_x.tolist(), zeros_y.tolist(), 'o')
plt.show()

print("zeros:", zeros_x, zeros_y)

In [ ]:
xt = torch.linspace(-1.3, 1.6, 8000, requires_grad=True)
yt = peaks(xt)
yt2 = yt.subtract(0.5).pow(2)
yt2.backward(torch.ones_like(xt))

dydx = xt.grad
print("derivatives:", dydx[:5])

y05_idx = ((dydx.abs() < 0.005) & (yt2 < 2e-7))
y05_x = xt[y05_idx]
y05_y = yt[y05_idx]

plt.plot(x, y)
plt.plot(y05_x.tolist(), y05_y.tolist(), 'o')
plt.show()

print("y=0.5:", y05_x, y05_y)

ANOTHER WAY of doing it

Very expensive to calculate function over large range

In [ ]:
xs = []
ys = []

x_ = torch.tensor(0.5, requires_grad=True)

y_ = peaks(x_)
xs.append(x_.item())
ys.append(y_.item())

y_.backward()
print(x_, y_, x_.grad)

x_ = x_ + 0.1 * x_.grad
x_.retain_grad()

y_ = peaks(x_)
xs.append(x_.item())
ys.append(y_.item())

# TODO: more steps

In [ ]:
plt.plot(x, y)
plt.scatter(xs, ys, marker='o', s=14, c='r')
plt.show()
x_.item(), y_.item()

# TODO : house price model